In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [2]:
google_reviews = pd.read_csv("../data/raw_data/google_reviews.csv")
kbh_facilities_reviews = pd.read_csv("../data/processed_data/kbh_facilities_reviews.csv")

In [5]:
# initialize map
lat, lng = 56, 9
denmark_map = folium.Map(location=[lat, lng], zoom_start=7)


# icons are from https://fontawesome.com/v4/
# choosing icons based on the activity type trying to get as close as we can so they can match
def get_icon(activity_type):
    icons = {
        "gym": "dumbbell", "fitness": "heartbeat", "outdoors": "tree",
        "nature": "leaf", "running": "running", "ball_sports": "users",
        "swimming": "swimmer", "yoga": "om", "dance": "music",
        "martial_arts": "fist-raised"
    }
    return icons.get(activity_type, "question-sign")

# Combined get_rating_average function
def get_rating_average(data, grouping_field):
    avg_rating_dict = data.groupby(grouping_field)['rating'].mean().to_dict()

    def get_color_for_location(key):
        avg_rating = avg_rating_dict.get(key, 0)
        if avg_rating >= 4:
            return 'green'
        elif avg_rating >= 2:
            return 'orange'
        else:
            return 'red'

    return get_color_for_location

# check if the location column is entirely null if so group by address instead
if kbh_facilities_reviews['location'].isnull().all():
    kbh_grouping_field = 'address'
else:
    kbh_grouping_field = 'location'

# group and aggregate kbh_facilities_reviews
kbh_grouped_data = kbh_facilities_reviews.groupby(kbh_grouping_field).agg({
    'lat': 'first', 'lng': 'first', 'activity': 'first', 
    'type': 'first', 'rating': 'mean'}).reset_index()

kbh_grouped_data['rating'] = kbh_grouped_data['rating'].round(1)

# group and aggregate google_reviews
google_grouped_data = google_reviews.groupby(['lat', 'lng']).agg({
    'type': 'first', 'rating': 'mean'}).reset_index()

google_grouped_data['rating'] = google_grouped_data['rating'].round(1)

# create a MarkerCluster
marker_cluster = MarkerCluster().add_to(denmark_map)

# Function to add markers from kbh_facilities_reviews
def add_kbh_markers(grouped_data):
    get_color = get_rating_average(grouped_data, kbh_grouping_field)
    for index, row in grouped_data.iterrows():
        lat, lng = row['lat'], row['lng']
        
        if lat is None or lng is None:
            continue
        try:
            
            lat_float = float(lat)
            lng_float = float(lng)
            
            color = get_color(row[kbh_grouping_field])
            icon = get_icon(row['type'])
            popup_content = f"Activity: {row['activity']}<br> Location: {row[kbh_grouping_field]}<br> Rating: {row['rating']}"
            marker_cluster.add_child(
                folium.Marker(
                    location=[lat_float, lng_float],
                    popup=popup_content,
                    icon=folium.Icon(color=color, icon=icon, prefix='fa')
                )
            )
        except ValueError:
            continue

# Function to add markers from google_reviews
def add_google_markers(grouped_data):
    
    get_color = get_rating_average(grouped_data, ('lat', 'lng'))
    for index, row in grouped_data.iterrows():
        
        lat, lng = row['lat'], row['lng']
        
        if lat is None or lng is None:
            continue
        try:
            
            lat_float = float(lat)
            lng_float = float(lng)
            
            color = get_color(lat_float, lng_float)
            popup_content = f"Type: {row['type']}<br> Rating: {row['rating']}"
            marker_cluster.add_child(
                folium.Marker(
                    location=[lat_float, lng_float],
                    popup=popup_content,
                    icon=folium.Icon(color=color, icon="dumbbell", prefix='fa')
                )
            )
        except ValueError:
            continue

# Add markers to the map
add_kbh_markers(kbh_grouped_data)
add_google_markers(google_grouped_data)

# Finalize and display the map
denmark_map.add_child(marker_cluster)
denmark_map


KeyError: ('lat', 'lng')